### Notebook for step-by-step data cleaning

In [33]:
import pandas as pd

df = pd.read_sas("data/raw/LLCP2015.XPT ", format='xport')

/opt/miniconda3/envs/522_group2/lib/python3.10/site-packages/pandas/io/sas/sas_xport.py:475: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[x] = v


In [34]:
df.head()


,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENUM,...,_PAREC1,_PASTAE1,_LMTACT1,_LMTWRK1,_LMTSCL1,_RFSEAT2,_RFSEAT3,_FLSHOT6,_PNEUMO2,_AIDTST3
0,1.0,1.0,b'01292015',b'01',b'29',b'2015',1200.0,2.015000e+09,2.015000e+09,1.0,...,4.0,2.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,1.0
1,1.0,1.0,b'01202015',b'01',b'20',b'2015',1100.0,2.015000e+09,2.015000e+09,1.0,...,2.0,2.0,3.0,3.0,4.0,2.0,2.0,NaN,NaN,2.0
2,1.0,1.0,b'02012015',b'02',b'01',b'2015',1200.0,2.015000e+09,2.015000e+09,1.0,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,NaN
3,1.0,1.0,b'01142015',b'01',b'14',b'2015',1100.0,2.015000e+09,2.015000e+09,1.0,...,4.0,2.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,9.0
4,1.0,1.0,b'01142015',b'01',b'14',b'2015',1100.0,2.015000e+09,2.015000e+09,1.0,...,4.0,2.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,1.0


In [35]:
# select relevant columns
# identified as relevant to diabetes prediction by Alex Teboul
relevant_columns = [
                    'DIABETE3',
                    '_RFHYPE5',  
                    'TOLDHI2',
                    '_CHOLCHK', 
                    '_BMI5', 
                    'SMOKE100', 
                    'CVDSTRK3',
                    '_MICHD', 
                    '_TOTINDA', 
                    '_FRTLT1',
                    '_VEGLT1', 
                    '_RFDRHV5', 
                    'HLTHPLN1',
                    'MEDCOST', 
                    'GENHLTH',
                    'MENTHLTH',
                    'PHYSHLTH',
                    'DIFFWALK', 
                    'SEX',
                    '_AGEG5YR',
                    'EDUCA',
                    'INCOME2' 
                    ]
df = df[relevant_columns]
df.head()

,DIABETE3,_RFHYPE5,TOLDHI2,_CHOLCHK,_BMI5,SMOKE100,CVDSTRK3,_MICHD,_TOTINDA,_FRTLT1,...,HLTHPLN1,MEDCOST,GENHLTH,MENTHLTH,PHYSHLTH,DIFFWALK,SEX,_AGEG5YR,EDUCA,INCOME2
0,3.0,2.0,1.0,1.0,4018.0,1.0,2.0,2.0,2.0,2.0,...,1.0,2.0,5.0,18.0,15.0,1.0,2.0,9.0,4.0,3.0
1,3.0,1.0,2.0,2.0,2509.0,1.0,2.0,2.0,1.0,2.0,...,2.0,1.0,3.0,88.0,88.0,2.0,2.0,7.0,6.0,1.0
2,3.0,1.0,1.0,1.0,2204.0,NaN,1.0,NaN,9.0,9.0,...,1.0,2.0,4.0,88.0,15.0,NaN,2.0,11.0,4.0,99.0
3,3.0,2.0,1.0,1.0,2819.0,2.0,2.0,2.0,2.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,2.0,9.0,4.0,8.0
4,3.0,1.0,2.0,1.0,2437.0,2.0,2.0,2.0,2.0,9.0,...,1.0,2.0,5.0,88.0,20.0,2.0,2.0,9.0,5.0,77.0


In [22]:
## CLEANING
# modifying reponses to make more sense 
# based on https://www.cdc.gov/brfss/annual_data/2015/pdf/codebook15_llcp.pdf

In [36]:
# Drop rows with NA values
df = df.dropna()

df.shape

(343606, 22)

In [37]:
# DIABETE3 (Diabetes)
# (Ever told) you have diabetes (some ordinality)
# we see some ordinality in the data, hence we will map the responses per the following:
# 1 (yes) --> 2 (yes)
# 2 (yes, during pregnancy) --> 0 (no)
# 3 (no) --> 0 (no)
# 4 (no, prediabetes) --> 1 (prediabetes)
# 7 (dont know) --> remove
# 9 (refused) --> remove
df['DIABETE3'] = df['DIABETE3'].replace({2:0, 3:0, 1:2, 4:1})
df = df[df.DIABETE3 != 7]
df = df[df.DIABETE3 != 9]

df.DIABETE3.unique()

array([0., 2., 1.])

In [38]:
# _RFHYPE5 (HighBP)
# Adults who have been told they have high blood pressure by a doctor, nurse, or other health professiona
# 1 (no) --> 0 (no)
# 2 (yes) --> 1 (yes)
# 9 (dont know) --> remove
df['_RFHYPE5'] = df['_RFHYPE5'].replace({1:0, 2:1})
df = df[df._RFHYPE5 != 9]

df._RFHYPE5.unique()

array([1., 0.])

In [39]:
# TOLDHI2 (HighChol)
# Have you EVER been told by a doctor, nurse or other health professional that your blood cholesterol is high?
# 1 (yes) --> 1 (yes)
# 2 (no) --> 0 (no)
# 7 (dont know) --> remove
# 9 (refused) --> remove
df['TOLDHI2'] = df['TOLDHI2'].replace({2:0})
df = df[df.TOLDHI2 != 7]
df = df[df.TOLDHI2 != 9]

df.TOLDHI2.unique()

array([1., 0.])

In [40]:
# _CHOLCHK (CholCheck)
# Cholesterol check within past five years
# 1 (Had cholesterol checked in past 5 years) --> 1
# 2 (Did not have cholesterol checked in past 5 years) --> 0 
# 3 (have never had cholesterol checked) --> 0
# 9 (don't know/not sure/refused/missing) --> remove
df['_CHOLCHK'] = df['_CHOLCHK'].replace({3:0,2:0})
df = df[df._CHOLCHK != 9]

df._CHOLCHK.unique()

array([1., 0.])

In [ ]:
# _BMI5 (BMI)
# Body Mass Index (BMI)
# no changes, just note that these are BMI * 100.
# So for example a BMI of 4018 is really 40.18)
df['_BMI5'] = df['_BMI5'].div(100).round(0)

df._BMI5.unique()

In [41]:
# SMOKE100 (Smoker)
# Have you smoked at least 100 cigarettes in your entire life?
# 1 (yes) --> 1
# 2 (no) --> 0
# 7 (dont knows) --> remove
# 9 (refused) --> remove
df['SMOKE100'] = df['SMOKE100'].replace({2:0})
df = df[df.SMOKE100 != 7]
df = df[df.SMOKE100 != 9]

df.SMOKE100.unique()

array([1., 0.])

In [42]:
# CVDSTRK3 (Stroke)
# (Ever told) you had a stroke.
# 1 (yes) -> 1
# 2 (no) -> 0
# 7 (dont knows) -> remove
# 9 (refused) -> remove
df['CVDSTRK3'] = df['CVDSTRK3'].replace({2:0})
df = df[df.CVDSTRK3 != 7]
df = df[df.CVDSTRK3 != 9]

df.CVDSTRK3.unique()

array([0., 1.])

In [43]:
# _MICHD (HeartDiseaseorAttack)
# Respondents that have ever reported having coronary heart disease (CHD) or myocardial infarction (MI)
# 1 (reported having) -> 1
# 2 (did not report having) -> 0
df['_MICHD'] = df['_MICHD'].replace({2: 0})

df._MICHD.unique()

array([0., 1.])

In [44]:
# _TOTINDA (PhysActivity)
# Adults who reported doing physical activity or exercise during the past 30 days other than their regular job
# 1 (had physical activity) -> 1
# 2 (No physical activity or exercise in last 30 days) -> 0 
# 9 (don't know/refused) -> remove
df['_TOTINDA'] = df['_TOTINDA'].replace({2:0})
df = df[df._TOTINDA != 9]

df._TOTINDA.unique()

array([0., 1.])

In [45]:
# _FRTLT1 (Fruits)
# Consume Fruit 1 or more times per day
# 1 (Consumed fruit one or more times per day) -> 1 
# 2 (Consumed fruit less than one time per day) -> 0 
# 9 (Don ́t know, refused or missing values) -> remove
df['_FRTLT1'] = df['_FRTLT1'].replace({2:0})
df = df[df._FRTLT1 != 9]

df._FRTLT1.unique()

array([0., 1.])

In [46]:
# _VEGLT1 (Veggies)
# Consume Vegetables 1 or more times per day
# 1 (Consumed vegetables one or more times per day) -> 1 
# 2 (Consumed vegetables less than one time per day) -> 0 
# 9 (Don ́t know, refused or missing values) -> remove
df['_VEGLT1'] = df['_VEGLT1'].replace({2:0})
df = df[df._VEGLT1 != 9]

df._VEGLT1.unique()

array([1., 0.])

In [47]:
# _RFDRHV5 (HvyAlcoholConsump)
# Heavy drinkers (adult men having more than 14 drinks per week and adult women having more than 7 drinks per week)
# 1 (no) -> 0
# 2 (yes) -> 1
# 9 (Don’t know/Refused/Missing) -> remove
df['_RFDRHV5'] = df['_RFDRHV5'].replace({1:0, 2:1})
df = df[df._RFDRHV5 != 9]

df._RFDRHV5.unique()

array([0., 1.])

In [48]:
# HLTHPLN1 (AnyHealthcare)
# Do you have any kind of health care coverage, including health insurance, prepaid plans such as HMOs, or government plans such as Medicare, or Indian Health Service?
# 1 (yes) -> 1
# 2 (no) -> 0
# 7 (Don’t know/Not Sure) -> remove
# 9 (Refused) -> remove
df['HLTHPLN1'] = df['HLTHPLN1'].replace({2:0})
df = df[df.HLTHPLN1 != 7]
df = df[df.HLTHPLN1 != 9]

df.HLTHPLN1.unique()

array([1., 0.])

In [49]:
# MEDCOST (NoDocbcCost)
# Was there a time in the past 12 months when you needed to see a doctor but could not because of cost?
# 1 (yes) -> yes
# 2 (no) -> no
# 7 (Don’t know/Not sure) -> remove
# 9 (refused) -> remvoe
df['MEDCOST'] = df['MEDCOST'].replace({2:0})
df = df[df.MEDCOST != 7]
df = df[df.MEDCOST != 9]

df.MEDCOST.unique()

array([0., 1.])

In [50]:
# GENHLTH (GenHlth)
# Would you say that in general your health is: (ordinal variable)
# 1 Excellent
# 2 Very good
# 3 Good
# 4 Fair
# 5 Poor
# 7 Don’t know/Not Sure -> remove
# 9 Refused -> remove
df = df[df.GENHLTH != 7]
df = df[df.GENHLTH != 9]

df.GENHLTH.unique()

array([5., 3., 2., 4., 1.])

In [55]:
# MENTHLTH (MentHlth)
# Now thinking about your mental health, which includes stress, depression, and problems with emotions, for how many days during the past 30 days was your mental health not good?
# 1-30 (number of days)
# 88 (None) -> 0 ; because it means none (no bad mental health days)
# 77 (Don’t know/Not sure) -> remove
# 99 (refused) -> remove
df['MENTHLTH'] = df['MENTHLTH'].replace({88:0})
df = df[df.MENTHLTH != 77]
df = df[df.MENTHLTH != 99]

df.MENTHLTH.unique()

array([18.,  0., 30.,  3.,  5., 15., 10.,  6., 20.,  2., 25.,  1., 29.,
        4.,  7.,  8., 21., 14., 26.,  9., 16., 28., 11., 12., 24., 17.,
       13., 23., 27., 19., 22.])

In [56]:
# PHYSHLTH (PhysHlth)
# Now thinking about your physical health, which includes physical illness and injury, for how many days during the past 30 days was your physical health not good?
# 1-30 (number of days)
# 88 (None) -> 0 ; because it means none (no bad physical health days)
# 77 (Don’t know/Not sure) -> remove
# 99 (refused) -> remove
df['PHYSHLTH'] = df['PHYSHLTH'].replace({88:0})
df = df[df.PHYSHLTH != 77]
df = df[df.PHYSHLTH != 99]

df.PHYSHLTH.unique()

array([15.,  0., 30.,  2., 14., 28.,  7., 20.,  3., 10.,  1.,  5., 17.,
        4., 19.,  6., 21., 12.,  8., 25., 27., 22., 29., 24.,  9., 16.,
       18., 23., 13., 26., 11.])

In [58]:
# DIFFWALK (DiffWalk)
# Do you have serious difficulty walking or climbing stairs?
# 1 (yes) -> yes
# 2 (no) -> no
# 7 (Don’t know/Not sure) -> remove
# 9 (refused) -> remvoe
df['DIFFWALK'] = df['DIFFWALK'].replace({2:0})
df = df[df.DIFFWALK != 7]
df = df[df.DIFFWALK != 9]

df.DIFFWALK.unique()

array([1., 0.])

In [59]:
# SEX (Sex)
# Indicate sex of respondent.
# in other words - is respondent male (somewhat arbitrarily chose this change because men are at higher risk for heart disease)
# 1 (Male) -> 1
# 2 (Female) -> 0
df['SEX'] = df['SEX'].replace({2:0})

df.SEX.unique()

array([0., 1.])

In [60]:
# _AGEG5YR (Age)
# Fourteen-level age category (ordinal)
# 1 (18-24yrs) : 5 year increments : 13 (80 and older).
# 14 (Don't know/Refused/Mising) -> remove
df = df[df._AGEG5YR != 14]

df._AGEG5YR.unique()

array([ 9.,  7., 11., 10., 13.,  8.,  4.,  6.,  2., 12.,  5.,  1.,  3.])

In [61]:
# EDUCA (Education)
# What is the highest grade or year of school you completed? (ordinal)
# 1 Never attended school or only kindergarten
# 2 Grades 1 through 8 (Elementary)
# 3 Grades 9 through 11 (Some high school)
# 4 Grade 12 or GED (High school graduate)
# 5 College 1 year to 3 years (Some college or technical school)
# 6 College 4 years or more (College graduate)
# 9 Refused -> remove
df = df[df.EDUCA != 9]

df.EDUCA.unique()

array([4., 6., 3., 5., 2., 1.])

In [62]:
# INCOME2 (Income)
# Is your annual household income from all sources: (ordinal)
# 1 Less than $10,000 
# 2 Less than $15,000 ($10,000 to less than $15,000) 
# 3 Less than $20,000 ($15,000 to less than $20,000) 
# 4 Less than $25,000 ($20,000 to less than $25,000)
# 5 Less than $35,000 ($25,000 to less than $35,000)
# 6 Less than $50,000 ($35,000 to less than $50,000)
# 7 Less than $75,000 ($50,000 to less than $75,000)
# 8 $75,000 or more
# 77 Don’t know/Not sure -> remove
# 99 Refused -> remove
df = df[df.INCOME2 != 77]
df = df[df.INCOME2 != 99]

df.INCOME2.unique()

array([3., 1., 8., 6., 4., 7., 2., 5.])

In [64]:
df.shape

(253680, 22)

In [65]:
df.groupby(['DIABETE3']).size()

DIABETE3
0.0    213703
1.0      4631
2.0     35346
dtype: int64

In [66]:
# rename columns for readability

df = df.rename(columns = {  
                            'DIABETE3':'Diabetes_012', 
                            '_RFHYPE5':'HighBP',  
                            'TOLDHI2':'HighChol',
                            '_CHOLCHK':'CholCheck', 
                            '_BMI5':'BMI', 
                            'SMOKE100':'Smoker', 
                            'CVDSTRK3':'Stroke',
                            '_MICHD':'HeartDiseaseorAttack', 
                            '_TOTINDA':'PhysActivity', 
                            '_FRTLT1':'Fruits',
                            '_VEGLT1':"Veggies", 
                            '_RFDRHV5':'HvyAlcoholConsump', 
                            'HLTHPLN1':'AnyHealthcare',
                            'MEDCOST':'NoDocbcCost', 
                            'GENHLTH':'GenHlth',
                            'MENTHLTH':'MentHlth',
                            'PHYSHLTH':'PhysHlth',
                            'DIFFWALK':'DiffWalk', 
                            'SEX':'Sex',
                            '_AGEG5YR':'Age',
                            'EDUCA':'Education',
                            'INCOME2':'Income' 
                            }
                )

df.head()

,Diabetes,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,4018.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,2509.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
3,0.0,1.0,1.0,1.0,2819.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
5,0.0,1.0,0.0,1.0,2652.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
6,0.0,1.0,1.0,1.0,2389.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0
